# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("../Output_Data/cities.csv")
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,84.20,79.0,40.0,13.80,EC,1.614711e+09
1,1,yellowknife,62.4560,-114.3525,-7.01,77.0,90.0,12.66,CA,1.614711e+09
2,2,hobart,-42.8794,147.3294,48.99,76.0,40.0,5.75,AU,1.614711e+09
3,3,deshnok,27.7986,73.3447,64.51,23.0,0.0,3.51,IN,1.614711e+09
4,4,albany,42.6001,-73.9662,24.01,30.0,0.0,5.01,US,1.614711e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Google developer API key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
#store lat and lng
locations = df[["Lat","Lng"]]
humidity = df["Humidity"]

#heatmap
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': 'auto'
}
heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2.2, layout=figure_layout)

# setting up layer's parameters
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=max(humidity),
                               point_radius = 2.8,)
# adding the layer
heatmap.add_layer(heat_layer)
# displaying heatmap
heatmap


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='auto', padding='1px', width='1000…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrowing down places with ideal weather conditions
temp_df = df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] >70) ]
weather_criteria = temp_df.loc[(temp_df["Wind Speed"] < 10) & (temp_df["Cloudiness"] == 0)]
weather_criteria


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,cape town,-33.9258,18.4232,71.01,82.0,0.0,8.05,ZA,1.614711e+09
61,64,wattegama,6.7989,81.4808,73.40,94.0,0.0,5.75,LK,1.614711e+09
74,78,ancud,-41.8697,-73.8203,75.20,41.0,0.0,6.91,CL,1.614711e+09
127,137,mecca,21.4267,39.8261,73.80,42.0,0.0,5.06,SA,1.614711e+09
157,168,banyo,6.7500,11.8167,74.73,22.0,0.0,4.23,CM,1.614711e+09
168,179,alice springs,-23.7000,133.8833,77.00,29.0,0.0,8.05,AU,1.614711e+09
276,299,allapalli,19.4167,80.0667,73.31,20.0,0.0,2.93,IN,1.614711e+09
300,324,pangani,-5.4333,38.9667,78.53,80.0,0.0,7.81,TZ,1.614711e+09
307,332,noumea,-22.2763,166.4572,78.01,88.0,0.0,4.61,NC,1.614711e+09
383,415,richards bay,-28.7830,32.0377,71.22,89.0,0.0,6.24,ZA,1.614711e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# copying our data frame into hotel_df and adding a column named "Hotel Name"
hotel_df = weather_criteria.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,cape town,-33.9258,18.4232,71.01,82.0,0.0,8.05,ZA,1.614711e+09,
61,64,wattegama,6.7989,81.4808,73.40,94.0,0.0,5.75,LK,1.614711e+09,
74,78,ancud,-41.8697,-73.8203,75.20,41.0,0.0,6.91,CL,1.614711e+09,
127,137,mecca,21.4267,39.8261,73.80,42.0,0.0,5.06,SA,1.614711e+09,
157,168,banyo,6.7500,11.8167,74.73,22.0,0.0,4.23,CM,1.614711e+09,
168,179,alice springs,-23.7000,133.8833,77.00,29.0,0.0,8.05,AU,1.614711e+09,
276,299,allapalli,19.4167,80.0667,73.31,20.0,0.0,2.93,IN,1.614711e+09,
300,324,pangani,-5.4333,38.9667,78.53,80.0,0.0,7.81,TZ,1.614711e+09,
307,332,noumea,-22.2763,166.4572,78.01,88.0,0.0,4.61,NC,1.614711e+09,
383,415,richards bay,-28.7830,32.0377,71.22,89.0,0.0,6.24,ZA,1.614711e+09,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",
    "radius": 5000,
    "keyword":"hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # getting city name
    city = row['City']
    # getting city coordinates from df
    params['location'] = f"{row['Lat']}, {row['Lng']} "
    
    # assemble url and make API request
    print(f"Processing request for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("There isn't any hotel in a 5000m radius.")
        
    print("--------------------------------------------------")

Processing request for Index 8: cape town.
Closest Hotel is 21 Nettleton.
--------------------------------------------------
Processing request for Index 61: wattegama.
There isn't any hotel in a 5000m radius.
--------------------------------------------------
Processing request for Index 74: ancud.
Closest Hotel is Hotel.
--------------------------------------------------
Processing request for Index 127: mecca.
Closest Hotel is Makkah Clock Royal Tower, A Fairmont Hotel.
--------------------------------------------------
Processing request for Index 157: banyo.
Closest Hotel is Hôtel Municipal de Banyo.
--------------------------------------------------
Processing request for Index 168: alice springs.
Closest Hotel is DoubleTree by Hilton Hotel Alice Springs.
--------------------------------------------------
Processing request for Index 276: allapalli.
Closest Hotel is Hotel IK Residency.
--------------------------------------------------
Processing request for Index 300: pangani.
C

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

<center><i><h1 style="background-color:black;color:orange">Map of Cities that meet the creteria for ideal weather conditions</h1></i></center>
<div style="background-color:LightSlateGray">
<ul>
    <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"> Using the previous created heat map, we will add the markes as a layer on top of the main layer, and this markers will have an info box content when clicked on </p>
    </li>
        <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"><span  style="color: yellow">Notes: </span> <span> Some of the cities don't have a hotel within a 5000m radius.(see list above)</span>   .</p>
    </li>
    </ul>
</div>

In [10]:
# Add marker layer ontop of heat map

# creating the markers
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

#adding the markers
heatmap.add_layer(markers)

# Display figure
heatmap


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='auto', padding='1px', width='1000…